In [15]:
# importando as bibliotecas iniciais
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [16]:
# carregando os dados
df = pd.read_csv('/datasets/users_behavior.csv')

In [17]:
# visualizando os tipos de dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [18]:
df.sample(10)

,calls,minutes,messages,mb_used,is_ultra
1307,19.0,112.69,12.0,4312.96,0
84,56.0,397.14,42.0,8449.53,0
286,45.0,314.64,14.0,7332.79,0
3140,64.0,467.46,20.0,15527.23,0
38,23.0,146.18,33.0,9711.45,0
2315,48.0,341.65,0.0,19000.02,0
2741,81.0,532.33,61.0,7653.43,1
951,0.0,0.00,185.0,31032.94,1
1064,46.0,345.98,0.0,20980.25,0
3179,43.0,305.21,10.0,26607.42,0


In [19]:
# calls e messages poderiam ser convertidos para int pois não hã meia ligação ou meia mensagem

# avaliando se haverá perda de dados na conversão do campos calls
np.array_equal(df["calls"],df["calls"].astype("int"))

True

Conclusion: No data loss expected in calls if we convert datatype to integer 

In [20]:
# convertendo calls para int
df["calls"] = df["calls"].astype("int")

In [21]:
# avaliando se haverá perda de dados na conversão do campos messages
np.array_equal(df["messages"],df["messages"].astype("int"))

True

Conclusion: No data loss expected in messages if we convert datatype to integer 

In [22]:
# convertendo messages para int
df["messages"] = df["messages"].astype("int")

In [23]:
# visualizando os tipos de dados novamente
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


In [24]:
# visualizando os dados
df.sample(10)

,calls,minutes,messages,mb_used,is_ultra
1134,17,138.90,10,3272.89,1
621,67,505.34,79,31408.79,1
799,83,519.66,52,15821.17,0
3002,35,213.87,15,20097.61,0
7,15,132.40,6,21911.60,0
205,95,672.78,15,3315.76,1
836,113,707.79,0,21008.04,0
1472,94,569.22,155,23422.79,1
1240,123,832.33,6,14670.49,1
574,11,77.94,28,25052.44,0


In [25]:
balance = df.groupby("is_ultra")["is_ultra"].count()
total = balance.sum()
percentages = (balance/total)
print(percentages)

is_ultra
0    0.693528
1    0.306472
Name: is_ultra, dtype: float64


Conclusion: We have about 70% of records in the Smart Plan and 30% on the Ultra Plan

In [26]:
# Divida os dados de origem em um conjunto de treinamento, um conjunto de validação e um conjunto de teste.

# estamos lidando com um problema de classificação, pois o plano sugerido só pode ser ultra ou smart.
# podemos, portanto testar os seguintes modelos: Arvore de Decisão, Floresta Aleatória e Regressão logística

In [27]:
# importando bibliotecas dos modelos
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [28]:
# criando conjunto de treinamento e validação
df_train,df_valid = train_test_split(df, test_size=.4, random_state=12345)

In [29]:
# dividindo o conjunto de validação em validação e teste
df_valid,df_test = train_test_split(df_valid,test_size =.5, random_state=12345)

In [30]:
# Preparando os dados nos 3 conjuntos

features_train = df_train.drop(["is_ultra"], axis=1)
target_train = df_train["is_ultra"]

features_valid = df_valid.drop(["is_ultra"],axis=1)
target_valid = df_valid["is_ultra"]

features_test = df_test.drop(["is_ultra"],axis=1)
target_test = df_test["is_ultra"]

In [31]:
# Modelo de Árvore de Decisão

# criando um loop para encontrar a max_depth ideal
best_acc = 0
best_depth = 0

for depth in range(1,11):

    # instanciando o modelo
    treemodel = DecisionTreeClassifier(random_state=12345, max_depth=depth)

    # treinando o modelo
    treemodel.fit(features_train,target_train)

    # testando previsões conjunto de validação
    valid_predictions = treemodel.predict(features_valid)
    
    # calculando accuracy conjunto de validação
    acc = accuracy_score(target_valid,valid_predictions)
    
    # identificando os melhores resultados conjunto de validação
    if acc > best_acc:
        best_acc = acc
        best_depth = depth

print("Acurácia do melhor modelo de árvore de decisão no conjunto de validação (max_depth = {}): {}".format(best_depth, best_acc))

Acurácia do melhor modelo de árvore de decisão no conjunto de validação (max_depth = 3): 0.7853810264385692


In [50]:
# Modelo de Árvore de Decisão
# otimizando hiperparametros na Arvore de Decisao

# importando gridsearch
from sklearn.model_selection import GridSearchCV

# Definindo os hiperparametros
param_grid = {
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10],
    'criterion': ['gini', 'entropy'],
    'max_depth': [2,3,5,7,10]
}

# Instanciando o modelo
dtc = DecisionTreeClassifier()

# instanciando o GridSearchCV
grid_search = GridSearchCV(estimator=dtc, param_grid=param_grid, cv=5, scoring='accuracy')

# Fazendo o fit
grid_search.fit(features_train, target_train)

# coletando resultados
print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)

Best Parameters:  {'criterion': 'entropy', 'max_depth': 7, 'min_samples_leaf': 10, 'min_samples_split': 5}
Best Score:  0.8086158401184308


In [51]:
# rodando o modelo de Árvore de Decisão novamente, agora com conjunto de teste e hiperparametros otimizados

# instanciando o modelo
treemodel = DecisionTreeClassifier(random_state=12345, criterion="entropy", max_depth=7, min_samples_leaf=10, min_samples_split=5 )

# treinando o modelo
treemodel.fit(features_train,target_train)

# testando previsões conjunto de teste
test_predictions = treemodel.predict(features_test)

# calculando accuracy conjunto de teste
acc_test = accuracy_score(target_test,test_predictions)

print("Acurácia do melhor modelo de árvore de decisão no conjunto de teste e hiperparametros otimizados):",acc_test)

Acurácia do melhor modelo de árvore de decisão no conjunto de teste e hiperparametros otimizados): 0.7916018662519441


In [45]:
# Modelo de Floresta Aleatória

# criando um laço para encontrar n_estimators ideal
best_score = 0
best_est = 0

for est in range(1, 16):
    # instanciando o modelo
    forestmodel = RandomForestClassifier(random_state=12345, n_estimators=est)
    
    # treinando o modelo
    forestmodel.fit(features_train,target_train)
    
    # calculando score
    score = forestmodel.score(features_valid,target_valid)
    
    # identificando melhores resultados
    if score > best_score:
        best_score = score
        best_est = est

print("Acurácia do melhor modelo de Floresta Aleatória no conjunto de validação (n_estimators = {}): {}".format(best_est, best_score))

Acurácia do melhor modelo de Floresta Aleatória no conjunto de validação (n_estimators = 12): 0.7869362363919129


In [46]:
# Modelo de Floresta Aleatória
# otimizando hiperparametros na Arvore de Decisao

# Definindo os hiperparametros
param_grid = {
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10],
    'n_estimators':[2, 5, 10, 12, 15]
}

# Instanciando o modelo
rfc = RandomForestClassifier()

# instanciando o GridSearchCV
rfcgrid_search = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5, scoring='accuracy')

# treinando o modelo
rfcgrid_search.fit(features_train, target_train)

# coletando resultados
print("Best Parameters: ", rfcgrid_search.best_params_)
print("Best Score: ", rfcgrid_search.best_score_)

Best Parameters:  {'max_features': 'sqrt', 'min_samples_leaf': 10, 'min_samples_split': 10, 'n_estimators': 10}
Best Score:  0.8195128187874301


In [52]:
# rodando o modelo de Floresta Aleatória com conjunto de teste e hiperparametros otimizados

# instanciando o modelo
forestmodel = RandomForestClassifier(random_state=12345, n_estimators=10, max_features="sqrt", min_samples_leaf=10, min_samples_split=10)

# treinando o modelo
forestmodel.fit(features_train,target_train)

# calculando score
score_test = forestmodel.score(features_test,target_test)

print("Acurácia do melhor modelo de Floresta Aleatória no conjunto de teste e hiperparametros otimizados:", score_test)

Acurácia do melhor modelo de Floresta Aleatória no conjunto de teste e hiperparametros otimizados: 0.7978227060653188


In [57]:
# Modelo de Regressão Logística
# otimizando hiperparametros na regressáo logistica

# Define the hyperparameter space
param_grid = {
    'C': [0.1, 1, 5, 10],
    'penalty': ['l1', 'l2'],
    'max_iter': [100, 500, 1000],
    'solver':['liblinear','saga']
}

# Initialize the LogisticRegression
lr = LogisticRegression()

# Initialize GridSearchCV
lrgrid_search = GridSearchCV(estimator=lr, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit the grid search
lrgrid_search.fit(features_train, target_train)

# Print the best hyperparameters and the corresponding score
print("Best Parameters: ", lrgrid_search.best_params_)
print("Best Score: ", lrgrid_search.best_score_)

/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: Convergenc

/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: Convergenc

/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: Convergenc

/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: Convergenc

Best Parameters:  {'C': 10, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}
Best Score:  0.7505268824439809


/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [60]:
# Modelo de Regressão Logística

# instanciando o modelo
model = LogisticRegression(random_state=12345, solver='liblinear', C=10, max_iter=100, penalty="l1")

# treinando o modelo
model.fit(features_train, target_train)

# calculando scores
score_valid = model.score(features_valid,target_valid)
score_test = model.score(features_test,target_test)

print("Acurácia do modelo de regressão logística no conjunto de validação:", score_valid)
print("Acurácia do modelo de regressão logística no conjunto de teste:", score_test)

Acurácia do modelo de regressão logística no conjunto de validação: 0.7558320373250389
Acurácia do modelo de regressão logística no conjunto de teste: 0.7387247278382582


Conclusion:

The best model is random forest with an accuracy of 0,7978, followed by decision tree with an accuracy of 0,7916 e logístic regression on third place with accuracy of 0,7387, all in the test group